In [1]:
# dev of cleverhans attack based on :
# https://github.com/tensorflow/cleverhans/blob/master/cleverhans_tutorials/mnist_tutorial_pytorch.py

In [2]:
from __future__ import print_function

In [3]:
import matplotlib as mpl
# set this 'backend' when using jupyter; do this before importing pyplot
mpl.use('nbagg')
import matplotlib.pyplot as plt

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

import sys
sys.path.append('/home/felix/Research/Adversarial Research/FGN---Research/')
import Finite_Gaussian_Network_lib as fgnl
import Finite_Gaussian_Network_lib.fgn_helper_lib as fgnh



In [5]:
!gpustat

countdown  Mon Aug 12 13:58:23 2019
[0] GeForce GTX 1080 | 39'C,   0 % |   613 /  8119 MB | felix(601M)
[1] GeForce GTX 1080 | 40'C,   0 % |   836 /  8118 MB |


In [6]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
use_cuda = True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
print("Using device:", device)

# manualy set cuda device
torch.cuda.set_device(0)

CUDA Available:  True
Using device: cuda


In [7]:
# MNIST dataset and dataloader declaration
# transforms does both the conversion from 0-255 to 0-1
# and normalizes by the precomputed mean and std

batch_size = 192

mnist_train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../MNIST-dataset', train=True, download=False, 
                   transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))
            ])), 
        batch_size=batch_size, shuffle=False)

mnist_test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../MNIST-dataset', train=False, download=False, 
                   transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))
            ])), 
        batch_size=batch_size, shuffle=False)

In [8]:
# model to test
model_path = "../Experiments/sample_models/sample_fgn_model_noisy_eval_p0.27_full.pth"
dict_path =  "../Experiments/sample_models/sample_fgn_model_noisy_eval_p0.27_state_dict.pth"
params_paths = "../Experiments/sample_models/sample_fgn_model_noisy_eval_p0.27_parameters.txt"

print(model_path)
# exp params
with open(params_paths) as f: 
    print(f.read())

../Experiments/sample_models/sample_fgn_model_noisy_eval_p0.27_full.pth
hidden_l_nums [32, 32]
drop_p 0.03125
ordinal 3.0
lambda for l2 loss 6.66666666667e-06
lambda for sigma loss 0.00666666666667
noisy_centers False
batch_size 192



In [9]:
# # full load
fgn_model = torch.load(model_path)
fgn_model.to(device)

# dict load
# fgn_model = fgnl.Feedforward_FGN_net(in_feats=28*28, out_feats=10, hidden_l_nums=[64,64], 
#                                      drop_p=0, noisy_centers=False).to(device)
# fgn_model.load_state_dict(state_dict=torch.load(dict_path))

Feedforward_FGN_net(
  (hidden_layers): ModuleList(
    (0): Dropout(p=0.03125)
    (1): FGN_layer()
    (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.03125)
    (4): FGN_layer()
    (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.03125)
  )
  (ib): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fl): FGN_layer()
)

In [10]:
# test the model
### Loss Functions for the FGN
fgn_cross_ent_loss = (lambda model,output,target:  F.cross_entropy(output,target.long()))

In [11]:
# set random eval to false to check 
fgn_model.set_random_eval(False)
fgn_test_res = fgnh.test(fgn_model, mnist_train_loader,
                        fgn_cross_ent_loss, pred_func=fgnh.cross_ent_pred_accuracy, verbose=True)

fgn_test_res = fgnh.test(fgn_model, mnist_test_loader,
                        fgn_cross_ent_loss, pred_func=fgnh.cross_ent_pred_accuracy, verbose=True)

Test set - Average loss: 0.2918, Accuracy: 54331/60000 (91%)
Test set - Average loss: 0.3059, Accuracy: 9040/10000 (90%)


In [12]:
# change to random eval
fgn_model.set_random_eval(True)
fgn_test_res = fgnh.test(fgn_model, mnist_train_loader,
                        fgn_cross_ent_loss, pred_func=fgnh.cross_ent_pred_accuracy, verbose=True)

fgn_test_res = fgnh.test(fgn_model, mnist_test_loader,
                        fgn_cross_ent_loss, pred_func=fgnh.cross_ent_pred_accuracy, verbose=True)

Test set - Average loss: 0.2914, Accuracy: 54273/60000 (90%)
Test set - Average loss: 0.3045, Accuracy: 9034/10000 (90%)


In [13]:
### start of tutorial code

In [14]:
### tutorial imports
from __future__ import absolute_import
from __future__ import division
# from __future__ import print_function
from __future__ import unicode_literals

import warnings
# import numpy as np
import tensorflow as tf
# import torch
# from torch import nn
# import torch.nn.functional as F
# from torch import optim
from torch.autograd import Variable
# from torchvision import datasets, transforms

from cleverhans.attacks import FastGradientMethod
# from cleverhans.compat import flags
from cleverhans.model import CallableModelWrapper
from cleverhans.utils import AccuracyReport
from cleverhans.utils_pytorch import convert_pytorch_model_to_tf

# FLAGS = flags.FLAGS

In [15]:
NB_EPOCHS = 6
BATCH_SIZE = 128
LEARNING_RATE = .001

In [16]:
nb_epochs=NB_EPOCHS
batch_size=BATCH_SIZE,
train_end=-1
test_end=-1
learning_rate=LEARNING_RATE

In [17]:
# We use tf for evaluation on adversarial data
sess = tf.Session()
x_op = tf.placeholder(tf.float32, shape=(None, 1, 28, 28,))

In [18]:
# Convert pytorch model to a tf_model and wrap it in cleverhans
torch_model = fgn_model
tf_model_fn = convert_pytorch_model_to_tf(torch_model)
cleverhans_model = CallableModelWrapper(tf_model_fn, output_layer='logits')

In [19]:
# Create an FGSM attack
fgsm_op = FastGradientMethod(cleverhans_model, sess=sess)
fgsm_params = {'eps': 0.3,
             'clip_min': 0.,
             'clip_max': 1.}
adv_x_op = fgsm_op.generate(x_op, **fgsm_params)
adv_preds_op = tf_model_fn(adv_x_op)

In [22]:
# Run an evaluation of our model against fgsm
total = 0
correct = 0
for xs, ys in mnist_test_loader:
    print(xs.shape, ys.shape, x_op.shape)
    adv_preds = sess.run(adv_preds_op, feed_dict={x_op: xs})
    correct += (np.argmax(adv_preds, axis=1) == ys.cpu().detach().numpy()).sum()
    total += test_loader.batch_size

torch.Size([192, 1, 28, 28]) torch.Size([192]) (?, 1, 28, 28)


InvalidArgumentError: assertion failed: [] [Condition x >= y did not hold element-wise:x (Placeholder:0) = ] [[[[-0.424212962 -0.424212962 -0.424212962...]]]...] [y (Cast/x:0) = ] [0]
	 [[node assert_greater_equal/Assert/AssertGuard/Assert (defined at /home/felix/.local/lib/python2.7/site-packages/cleverhans/utils_tf.py:624)  = Assert[T=[DT_STRING, DT_STRING, DT_FLOAT, DT_STRING, DT_FLOAT], summarize=3, _device="/job:localhost/replica:0/task:0/device:CPU:0"](assert_greater_equal/Assert/AssertGuard/Assert/Switch, assert_greater_equal/Assert/AssertGuard/Assert/data_0, assert_greater_equal/Assert/AssertGuard/Assert/data_1, assert_greater_equal/Assert/AssertGuard/Assert/Switch_1, assert_greater_equal/Assert/AssertGuard/Assert/data_3, assert_greater_equal/Assert/AssertGuard/Assert/Switch_2/_15)]]
	 [[{{node PyFunc/_5}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnati